*The script to create the submission is in the end of the file.

In [716]:
import pandas as pd
import numpy as np

# Reading train and test files

In [717]:
train = pd.read_csv('Data/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [718]:
test = pd.read_csv('Data/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Initial Logistic Regression Model with only numerical columns


The "gender_submission" provided by Kaggle is a submission which predict that every woman survives. The accuracy of this submission is 0.76555. 

The Logistic Regression Model below got an accuracy of 0.66507.


In [719]:
from sklearn.linear_model import LogisticRegression
num_col = ["PassengerId","Pclass","Age","SibSp","Parch","Fare"]
X = train[num_col].fillna(-1)
y = train["Survived"].fillna(-1)

In [720]:
model = LogisticRegression()
model.fit(X,y)
p = model.predict(test[num_col].fillna(-1))

/home/jardel/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Feature engineering

In [721]:
#creating 0 or 1 columns to get information from some categorical data
train["bool_sex"] = (train["Sex"] == "male").astype(int)
train["s_embarked"] = (train["Embarked"] == "S").astype(int)
train["c_embarked"] = (train["Embarked"] == "C").astype(int)
test["bool_sex"] = (test["Sex"] == "male").astype(int)
test["s_embarked"] = (test["Embarked"] == "S").astype(int)
test["c_embarked"] = (test["Embarked"] == "C").astype(int)


In [722]:
#get all different titles from names (like Mr.or Miss.)

train_titles = []
test_titles = []
for name in train["Name"]:
    if name.split()[1][-1] == '.':
        train_titles.append(name.split()[1])
    else:
        train_titles.append(name.split()[2])
for name in test["Name"]:
    if name.split()[1][-1] == '.':
        test_titles.append(name.split()[1])
    else:
        test_titles.append(name.split()[2])


print(pd.Series(train_titles).unique())
print(pd.Series(test_titles).unique())

['Mr.' 'Mrs.' 'Miss.' 'Master.' 'Don.' 'Rev.' 'hoef,' 'Dr.' 'Castellana,'
 'Mme.' 'Ms.' 'Major.' 'Manent,' 'Lady.' 'Sir.' 'Mlle.' 'Col.' 'Capt.'
 'Countess.' 'Jonkheer.' 'More,']
['Mr.' 'Mrs.' 'Miss.' 'Master.' 'Rodriguez,' 'Ms.' 'Col.' 'Rev.' 'More,'
 'Dr.' 'Castello,' 'Ocana,']


In [723]:
#creating features for every title
titles=[]
for title in list(pd.Series(train_titles).unique()):
    if title[-1]==".":
        titles.append(title)

for title in titles:
    train[title] = train["Name"].apply(lambda x: x.split()[1]).str.contains(title).astype(int)
    test[title] = test["Name"].apply(lambda x: x.split()[1]).str.contains(title).astype(int)
    

In [724]:
print("Train: ")
for col in train.columns:
    if train[col].isnull().values.any():
        print(col, train[col].isnull().sum())
      
print("\nTest:")
for col in test.columns:
    if test[col].isnull().values.any():
        print(col, test[col].isnull().sum())

#fill na with mean
train["Age"]= train["Age"].fillna(np.mean(train["Age"]))
test["Age"] = test["Age"].fillna(np.mean(train["Age"]))
test["Fare"] = test["Fare"].fillna(np.mean(test["Fare"]))
train["Cabin"] = train["Cabin"].fillna("U")
test["Cabin"] = test["Cabin"].fillna("U")

Train: 
Age 177
Cabin 687
Embarked 2

Test:
Age 86
Fare 1
Cabin 327


In [725]:
#cabin feature
train["Cabin"] = train["Cabin"].apply(lambda x: x[0])
test["Cabin"] = test["Cabin"].apply(lambda x: x[0])

#train["Cabin"].unique()
#test["Cabin"].unique()

#initializing cabin columns
cabincol = []
for cabin in train["Cabin"].unique():
    cabincol.append("cabin_"+cabin)   
 
for col_name in cabincol:
    train[col_name] = (train["Cabin"] == col_name[-1]).astype(int)
    test[col_name] = (test["Cabin"] == col_name[-1]).astype(int)


# Validation

In [726]:
num_col = ["Pclass","Age","SibSp","Parch","Fare","bool_sex","s_embarked",
           "c_embarked"]
num_col = num_col + titles + cabincol
X = train[num_col]
y = train["Survived"]

In [727]:
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier

In [745]:
results = []
kf = RepeatedKFold(n_splits=2,n_repeats=10,random_state=10)

for train_l, valid_l in kf.split(X):
    X_train, X_valid = X.iloc[train_l],X.iloc[valid_l]
    y_train, y_valid = y.iloc[train_l],y.iloc[valid_l]
    
    model = LogisticRegression()
    model.fit(X_train,y_train)
    p = model.predict(X_valid)
    
    acc = np.mean(y_valid == p)
    results.append(acc)

/home/jardel/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/jardel/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

/home/jardel/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/jardel/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

In [746]:
np.mean(results)

0.8218788733813674

# Final model

In [747]:
from sklearn.linear_model import LogisticRegression

In [750]:
model = LogisticRegression()
model.fit(X,y)
p = model.predict(test[num_col])

/home/jardel/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Create submission

In [751]:
sub = pd.Series(p,index=test["PassengerId"],name="Survived")
sub.to_csv("submission.csv",header=True)